In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.kernel_ridge import KernelRidge
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
#os.listdir('drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction')
folderdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/'
traindir = '/content/drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train.csv'
submissiondir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/sample_submission.csv'
testfolderdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/test'
train_X_npdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train_X.npy'
train_Y_npdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train_Y.npy'
train_X_mapdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train_X.memmap'
train_Y_mapdir = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train_Y.memmap'
train_X_csv = 'drive/My Drive/MATH 189/Final Project/LANL-Earthquake-Prediction/train_X_1.csv'

In [0]:
train_size = 629145480
train_X = np.memmap(train_X_mapdir, dtype=np.int16, mode='r', shape=(train_size,))
train_Y = np.memmap(train_Y_mapdir, dtype=np.float64, mode='r', shape=(train_size,))

In [0]:
df = pd.DataFrame({
    'x_train' : train_X, 
    'y_train' : train_Y
})

In [0]:
## Creating Segment Zones for the data 
row_count = 150000
segments = int(train_size/row_count)


# Creating empty feature dataframes

x_train = pd.DataFrame(index = range(segments), dtype = np.float64)
y_train = pd.DataFrame(index = range(segments), dtype = np.float64, columns = ['rate_of_time_to_fail', 'avg_time_to_fail', 'last_time_to_fail'])

In [0]:
for segment in range(segments):
  segment_df = df.iloc[segment * row_count: segment*row_count + row_count]
  x_vals = pd.Series(segment_df['x_train'])
  y_vals = pd.Series(segment_df['y_train'])
  
  x_train.loc[segment, 'mean'] = x_vals.mean()
  x_train.loc[segment, 'standard_dev'] = x_vals.std()
  x_train.loc[segment, 'max'] = x_vals.max()
  x_train.loc[segment, 'min'] = x_vals.min()
  x_train.loc[segment, 'kurtosis'] = x_vals.kurt()
  
    
  x_train.loc[segment, 'avg_rolling_mean'] = x_vals.rolling(1000).mean().mean()
  x_train.loc[segment, 'avg_rolling_std'] = x_vals.rolling(1000).std().mean()
  
  x_train.loc[segment, 'std_rolling_mean'] = x_vals.rolling(1000).mean().std()
  x_train.loc[segment, 'std_rolling_std'] = x_vals.rolling(1000).std().std()
  
  y_train.loc[segment, 'avg_time_to_fail'] = y_vals.mean()
  y_train.loc[segment, 'rate_of_time_to_fail'] = np.mean(np.diff(y_vals))
  y_train.loc[segment, 'last_time_to_fail'] = y_vals.values[-1]

In [0]:
ridge_cv = RidgeCV(alphas = [.0001,.001,.01,.1,1,10], cv = 5)

In [58]:
ridge_cv.fit(x_train,y_train['last_time_to_fail'])

RidgeCV(alphas=array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]), cv=5,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [59]:
ridge_cv.score(x_train,y_train['last_time_to_fail'])

0.36754369791049424

In [0]:
kernel_regression = KernelRidge(alpha = [.0001,.001,.01,.1,1,10], kernel = "rbf", gamma = .025)

In [55]:
kernel_regression.fit(x_train, y_train['last_time_to_fail'])

KernelRidge(alpha=[0.0001, 0.001, 0.01, 0.1, 1, 10], coef0=1, degree=3,
      gamma=0.025, kernel='rbf', kernel_params=None)

In [56]:
kernel_regression.score(x_train, y_train['last_time_to_fail'])

0.884460322024878

In [62]:
x_train

,mean,standard_dev,max,min,kurtosis,avg_rolling_mean,avg_rolling_std,std_rolling_mean,std_rolling_std
0,4.884113,5.101106,5.101106,-98.0,33.662481,4.883418,4.288590,0.295716,2.769782
1,4.725767,6.588824,6.588824,-154.0,98.758517,4.724876,4.843486,0.231587,4.492920
2,4.906393,6.967397,6.967397,-106.0,33.555211,4.905840,5.423013,0.267013,4.402155
3,4.902240,6.922305,6.922305,-199.0,116.548172,4.901486,4.939280,0.266701,4.873539
4,4.908720,7.301110,7.301110,-126.0,52.977905,4.910196,5.121868,0.228006,5.213382
5,4.913513,5.434111,5.434111,-144.0,50.215147,4.914758,4.538771,0.293942,2.988892
6,4.855660,5.687823,5.687823,-78.0,23.173004,4.856829,4.799256,0.219473,3.076807
7,4.505427,5.854512,5.854512,-134.0,52.388738,4.504153,4.503100,0.218515,3.763111
8,4.717833,7.789643,7.789643,-156.0,65.360261,4.718397,5.695230,0.240917,5.328307
9,4.730960,6.890459,6.890459,-126.0,53.760207,4.731377,5.209570,0.258463,4.536904
